# 银行产品评论观点提取
**南京大学计算机科学与技术系研究生选修课程《自然语言处理》（2021秋）课程项目作业**

【2022年1月16日重大发现】惊悉本项目作业就是照搬的2021年第九届CCF大数据与计算智能大赛的赛题“产品评论观点提取”([链接](https://www.datafountain.cn/competitions/529))，官方也已在AI Studio平台给出[基线版本](https://aistudio.baidu.com/aistudio/projectdetail/2417709)，可在大赛官网下载数据集，也可在AI Studio平台搜索“产品评论观点提取”获取更多项目源码参考。也已有其他同学在AI Studio[公开了源码](https://aistudio.baidu.com/aistudio/projectdetail/3210928)。心累...

本项目可在[百度AI Studio平台](https://aistudio.baidu.com/aistudio/projectdetail/3281489)在线运行和调试（需要使用GPU环境）。项目源码已上传[Github](https://github.com/MilesPoupart/nju-nlp-project-2021)。

项目数据集已包含在本项目`/data/data122751`路径中，具体数据集说明和下载见[数据集详情页面](https://aistudio.baidu.com/aistudio/datasetdetail/122751)。

本项目的代码主要参考了[这一博文](https://blog.csdn.net/weixin_41611054/article/details/118487666)（[GitHub项目地址](https://github.com/zhenhao-huang/paddlehub_ernie_emotion_analysis)），特表示感谢！

## 项目要求
现有有关银行及银行相关产品的若干条中文评论，要求对这些评论进行观点提取。具体而言，分为以下两个子任务：

### 实体识别
要求识别出原始评论文本中的实体及类型，并按BIO格式进行标注。<br>需要进行识别的实体有：银行、产品、用户评论中的名词及形容词，具体的标注标签及说明如下表所示：

|标签|说明|
|:----:|:----:|
|B-BANK|代表银行实体的开始|
|I-BANK|代表银行实体的内部|
|B-PRODUCT|代表产品实体的开始|
|I-PRODUCT|代表产品实体的内部|
|B-COMMENTS_N|代表用户评论（名词）|
|I-COMMENTS_N|代表用户评论（名词）实体的内部|
|B-COMMENTS_ADJ|代表用户评论（形容词）|
|I-COMMENTS_ADJ|代表用户评论（形容词）实体的内部|
|O|代表不属于标注的范围|

下图是一个标注的例子：
![实体识别的标注举例](https://ai-studio-static-online.cdn.bcebos.com/4966caab43894ab8ad1a7a85bdaff87a8437b4bf20b74ab0ad379c7cdcf1a248)

### 情感分类
根据用户评论的文本内容，判断其情感极性，并对其情感进行分类。<br>本次实验任务中，需要将用户的评论划分为正面（1）、负面（0）和中立（2）三种类型。

## 数据预处理
本节代码主要对应to_tsv.py
### 导入原始数据

分别读入两个任务需要的数据，并将数据进行打乱。

整个数据集共有6022条评论数据。

In [1]:
import pandas as pd
file_path = "data/data122751/train.csv"
classify_rawtext = pd.read_csv(file_path, sep=",", usecols=['class', 'text'])
classify_rawtext = classify_rawtext.sample(frac=1)  # 打乱数据集
print(len(classify_rawtext))
ner_text = pd.read_csv(file_path, sep=",", usecols=['text', 'BIO_anno'])
ner_text = ner_text.sample(frac=1)  # 打乱数据集

6022


### 处理情感分类数据
根据PaddleHub对[文本分类自定义数据的要求](https://paddlehub.readthedocs.io/zh_CN/release-v2.1/finetune/customized_dataset.html#id11)进行处理。
+ text和class的列位置，使得class列在前，符合输入要求
+ 按95%：5%划分训练集和验证集，充分利用训练数据。这里划分的测试集与验证集完全相同，仅用于程序计算相关指标。
+ 整个数据集分类标签为0、1、2（情感分类为负面、正面、中立）的比例分别大致为10%、4%、86%


In [2]:
gen_classify = True  # 控制是否生成情感分类训练数据

if gen_classify:
    # 交换列位置
    cols = list(classify_rawtext)
    print(cols)
    cols.insert(0, cols.pop(cols.index('class')))
    print(cols)
    classify_text = classify_rawtext.loc[:, cols]

    # 划分训练集、验证集、测试集
    classify_train = classify_text[:int(len(classify_text) * 0.95)]
    # classify_text[int(len(classify_text) * 0.8):int(len(classify_text) * 0.9)]
    classify_dev = classify_text[int(len(classify_text) * 0.95):]
    classify_test = classify_text[int(len(classify_text) * 0.95):]
    # 分别保存为tsv文件
    classify_train.to_csv('data/data122751/classify_train_data.tsv',
                          sep='\t', header=None, index=False, columns=None, mode="w")
    classify_dev.to_csv('data/data122751/classify_dev_data.tsv',
                        sep='\t', header=None, index=False, columns=None, mode="w")
    classify_test.to_csv('data/data122751/classify_test_data.tsv',
                         sep='\t', header=None, index=False, columns=None, mode="w")

    # 验证训练集和测试集标签分布是否均匀
    for file in ['classify_train_data', 'classify_dev_data', 'classify_test_data']:
        file_path = f"data/data122751/{file}.tsv"
        text = pd.read_csv(file_path, sep="\t", header=None)
        prob = dict()
        total = len(text[0])
        for i in text[0]:
            if prob.get(i) is None:
                prob[i] = 1
            else:
                prob[i] += 1
        # 按标签排序
        prob = {i[0]: round(i[1] / total, 3)
                for i in sorted(prob.items(), key=lambda k: k[0])}
        print(file, '\t', prob, total)

['text', 'class']
['class', 'text']
classify_train_data 	 {0: 0.106, 1: 0.038, 2: 0.856} 5720
classify_dev_data 	 {0: 0.093, 1: 0.043, 2: 0.864} 302
classify_test_data 	 {0: 0.093, 1: 0.043, 2: 0.864} 302


### 处理实体标注数据
根据PaddleHub对[序列标注自定义数据的要求](https://paddlehub.readthedocs.io/zh_CN/release-v2.1/finetune/customized_dataset.html#id16)进行处理。

在实体标注的数据集中插入分隔符，方便模型进行后续的分字处理，并将label与token一一对应。例如以“/”作为分隔符，可以将text=“招行15.0W”，labels=“B-BANK I-BANK O O O O O”的一组数据处理成为text=“招/行/1/5/./0/W”，labels=“B-BANK/I-BANK/O/O/O/O/O”。

通过检查数据集中没有出现过的特殊符号，可以发现仅有少数符号如\#、￥、|、 \等没有在数据集的文本中出现，不会引起错误的分割。

但在后续的实验中会发现，上述的这些符号在tokenize阶段都会转换为[UNK]，且模型在最后预测时无法处理已插入分隔符的文本，会对分隔符也生成一个预测标签（大多数情况下是O），进而导致预测结果极度混乱。这说明如果无法对分隔符号进行tokenize，有可能在处理时无法识别分隔符并对其正确处理。(*若要验证这一观察，可将split_char进行修改，并在预测时输出部分结果进行观察*)

在参考[相关官方文档中序列标注和命名实体识别的样例代码](https://paddlehub.readthedocs.io/zh_CN/release-v2.1/finetune/sequence_labeling.html)所采用的数据集[MSRA NER](https://bj.bcebos.com/paddlehub-dataset/msra_ner.tar.gz)后，可以发现这一数据集中的分隔符是'\002'，经测试，这一分隔符可以使得相关接口调用后正常识别并分字。因此，最终选用'\002'作为分隔符对序列标注的数据进行预处理。

同样按95%：5%划分训练集和验证集，充分利用训练数据。这里划分的测试集与验证集完全相同，仅用于程序计算相关指标。


In [3]:
gen_ner = True  # 控制是否生成实体识别数据
split_char = "\002"  # 分隔符
if gen_ner:
    sentence_cnt = len(ner_text)
    for i in range(sentence_cnt):
        raw_sentence = ner_text["text"][i].lower()  # 全部转小写
        if i <= 2:
            print("Raw text:\t%s" % raw_sentence)
        raw_sentence = split_char.join(raw_sentence)  # 文本逐字插入分隔符
        if i <= 2:
            print("Split text:\t%s" % raw_sentence)
        ner_text.loc[i, "text"] = raw_sentence  # 保存结果

        raw_lable = ner_text["BIO_anno"][i]
        if i <= 2:
            print("Raw label:\t%s" % raw_lable)
        raw_lable = raw_lable.replace(" ", split_char)  # 标签间插入分隔符
        if i <= 2:
            print("Split label:\t%s" % raw_lable)
            print("")
        ner_text.loc[i, "BIO_anno"] = raw_lable  # 保存结果

    # 划分训练集验证集测试集
    ner_train = ner_text[:int(len(ner_text) * 0.95)]
    # ner_text[int(len(ner_text) * 0.8):int(len(ner_text) * 0.9)]
    ner_dev = ner_text[int(len(ner_text) * 0.95):]
    ner_test = ner_text[int(len(ner_text) * 0.95):]

    # 保存到文件
    ner_train.to_csv('data/data122751/ner_train_data.tsv',
                     sep='\t', header=None, index=False, columns=None, mode="w")
    ner_dev.to_csv('data/data122751/ner_dev_data.tsv', sep='\t',
                   header=None, index=False, columns=None, mode="w")
    ner_test.to_csv('data/data122751/ner_test_data.tsv', sep='\t',
                    header=None, index=False, columns=None, mode="w")

Raw text:	交行14年用过，半年准备提额，却直接被降到1ｋ，半年期间只t过一次三千，其它全部真实消费，第六个月的时候为了增加评分提额，还特意分期两万，但降额后电话投诉，申请提...
Split text:	交行14年用过，半年准备提额，却直接被降到1ｋ，半年期间只t过一次三千，其它全部真实消费，第六个月的时候为了增加评分提额，还特意分期两万，但降额后电话投诉，申请提...
Raw label:	B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N O O O O O B-COMMENTS_ADJ I-COMMENTS_ADJ O O O O O O O O O O O O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N O O B-COMMENTS_N I-COMMENTS_N O O O O B-PRODUCT I-PRODUCT O O O O B-COMMENTS_ADJ O O O O O O O O O O O O O
Split label:	B-BANKI-BANKOOOOOOOOOOB-COMMENTS_NI-COMMENTS_NOOOOOB-COMMENTS_ADJI-COMMENTS_ADJOOOOOOOOOOOOOOOOOOOOOB-COMMENTS_NI-COMMENTS_NOOOOOOOOOOB-COMMENTS_NI-COMMENTS_NOOB-COMMENTS_NI-COMMENTS_NOOOOB-PRODUCTI-PRODUCTOOOOB-COMMENTS_ADJOOOOOOOOOOOOO

Raw text:	单标我有了，最近visa双标返现活动好
Split text:	单标我有了，最近

### 处理最终测试集
将测试集也转换为tsv格式，情感分类的数据文件直接转换即可，实体识别的数据需要插入分隔符。

In [4]:
gen_final = True  # 控制是否生成最终测试集
if gen_final:
    # 生成情感分类任务的测试集
    finaltest = pd.read_csv("data/data122751/test.csv", sep=",")
    finaltest.to_csv('data/data122751/classify_finaltest_data.tsv',
                     sep='\t', header=None, index=False, columns=None, mode="w")
    # 生成实体标注任务的测试集
    for i in range(len(finaltest)):
        raw_sentence = finaltest["text"][i]
        raw_sentence = split_char.join(raw_sentence)  # 插入分隔符
        finaltest.loc[i, "text"] = raw_sentence
    finaltest.to_csv('data/data122751/ner_finaltest_data.tsv',
                     sep='\t', header=None, index=False, columns=None, mode="w")

## 情感分类
### 导入数据集
根据[PaddleHub文档的要求](https://paddlehub.readthedocs.io/zh_CN/release-v2.1/finetune/customized_dataset.html#id15),需要继承基类TextClassificationDataset。按要求修改即可。


In [17]:
import paddle
import paddlehub as hub
import ast
import argparse
from paddlehub.datasets.base_nlp_dataset import TextClassificationDataset


class MyDataset(TextClassificationDataset):
    # 数据集存放目录
    base_path = 'data/data122751'
    # 数据集的标签列表，多分类标签格式为['0', '1', '2', '3',...]
    label_list = ['0', '1', '2']

    def __init__(self, tokenizer, max_seq_len: int = 128, mode: str = 'train'):
        if mode == 'train':
            data_file = 'classify_train_data.tsv'
        elif mode == 'test':
            data_file = 'classify_test_data.tsv'
        else:
            data_file = 'classify_dev_data.tsv'
        super().__init__(
            base_path=self.base_path,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            data_file=data_file,
            label_list=self.label_list,
            is_file_with_header=False)

### 设置参数并训练
使用Adam优化器，相关默认设置见[文档说明](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/optimizer/Adam_cn.html)。

如要重新训练，请务必**指定一空目录保存参数**，或**删除默认的`ernie_checkpoint_classify`目录**，否则程序会用上一个epoch的参数继续训练！！！

In [18]:
# 参数设置
num_epoch = 3  # Number of epoches for fine-tuning.
use_gpu = True  # Whether use GPU for fine-tuning, input should be True or False
learning_rate = 5e-5  # Learning rate used to train with warmup.
max_seq_len = 128  # Number of words of the longest seqence.
batch_size = 32  # Total examples' number in batch for training.
checkpoint_dir = './ernie_checkpoint_classify'  # Directory to model checkpoint
save_interval = 1  # Save checkpoint every n epoch.

# 选择模型、任务和类别数
model = hub.Module(name='ernie', task='seq-cls',
                   num_classes=len(MyDataset.label_list))

train_dataset = MyDataset(tokenizer=model.get_tokenizer(),
                          max_seq_len=max_seq_len, mode='train')
dev_dataset = MyDataset(tokenizer=model.get_tokenizer(),
                        max_seq_len=max_seq_len, mode='dev')
test_dataset = MyDataset(tokenizer=model.get_tokenizer(),
                         max_seq_len=max_seq_len, mode='test')

optimizer = paddle.optimizer.Adam(
    learning_rate=learning_rate, parameters=model.parameters())
trainer = hub.Trainer(
    model, optimizer, checkpoint_dir=checkpoint_dir, use_gpu=use_gpu)
trainer.train(train_dataset, epochs=num_epoch, batch_size=batch_size,
              eval_dataset=dev_dataset, save_interval=save_interval)
# 在测试集上评估当前训练模型
trainer.evaluate(test_dataset, batch_size=batch_size)

[2022-01-15 17:02:56,617] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-01-15 17:02:58,411] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-01-15 17:02:59,651] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-01-15 17:02:59,728] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-01-15 17:02:59,808] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...
[2022-01-15 17:03:02,012] [   TRAIN] - Epoch=1/3, Step=10/179 loss=0.8716 acc=0.6250 lr=0.000050 step/sec=4.54 | ETA 00:01:58
[2022-01-15 17:03:04,166] [   TRAIN] - Epoch=1/3, Step=20/179 loss=0.4965 acc=0.8594 lr=0.000050 step/sec=4.64 | ETA 00:01:56
[2022-01-15 17:03:06,318] [   TRAIN] - Epoch=1/3, Step=30/179 loss=0.5548 acc=0.8187 lr=0.000050 step/sec=4.65 | ETA 00:01:56
[2022-01-15 17:03:08,468] [   TRAIN] - Epoch=1/3, Step=40/179 loss=0.4147 acc=0.8781 lr=0.000050 step/sec

{'metrics': defaultdict(int, {'acc': 0.8774834437086093})}

以上代码块如有运行问题，可直接运行funetune_ernie-classify.py源文件

### 结果预测
如要对验证集的进行预测并对其指标进行计算，可将final_predict改为`False`。

我在本任务的训练中得到的最好的模型参数保存在`/ernie_checkpoint_classify_9505_2_acc93.71/best_model/model.pdparams`（由于AI Studio平台限制，公开文件大小不超过1G，参数文件可能会在其他平台公开）


In [19]:
final_predict = True
if final_predict:
    file_path = "data/data122751/classify_finaltest_data.tsv"
else:
    file_path = "data/data122751/classify_test_data.tsv"

text = pd.read_csv(file_path, sep="\t", header=None)
label_map = {0: 0, 1: 1, 2: 2}  # {0: 'negative', 1: 'positive', 2:'medium'}
data = [[i] for i in text[1]]

model = hub.Module(name='ernie', task='seq-cls',
                   load_checkpoint='./ernie_checkpoint_classify/best_model/model.pdparams', label_map=label_map)
results = model.predict(data, max_seq_len=128, batch_size=1, use_gpu=True)

[2022-01-15 17:07:50,465] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-01-15 17:07:55,722] [    INFO] - Loaded parameters from /home/aistudio/ernie_checkpoint_classify/best_model/model.pdparams
[2022-01-15 17:07:55,843] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


输出部分结果并保存结果/计算验证集指标。以下代码块仅实现了对准确率的计算，未计算项目评测使用的$\kappa$指标，如有需要，可自行添加。 

In [20]:
# print(results)
for i in range(10):
    print(results[i], data[i])
if final_predict:
    resultsDF = pd.DataFrame(data=results)
    resultsDF.to_csv('classify_result.csv', sep='\t',
                     header=None, index=False, columns=None, mode="w")
else:
    # 输出测试集准确率
    count = 0
    for i, j in zip(text[0], results):
        # print(type(i), type(j))
        if int(i) == int(j):
            count += 1
    print("测试集准确率:", count / len(results))

2 ['不能??好像房贷跟信用卡是分开审核的反正我的不得']
2 ['我感觉这样才是合理的，花呗白条没要那么多信息，照样可以给额度。有征信威慑，没那么多人敢借了不还。与其眉毛胡子一把抓，还不如按额度区间对客户进行不同程度的调查，免...']
0 ['这几天也接到了建行的分期电话，让我1.6分12期，跟他说5000，3期，意思意思。快两年了还没首提，心塞']
2 ['我是建行贷款，别的银行信用卡没还，账单大概4w']
0 ['银行耍无赖啊']
2 ['晒晒“断”卡，我的招行卡猫咪卡']
2 ['交行_197k\u3000民生_217k\u3000中行_125k\u3000工行_158k']
2 ['工商84000']
2 ['201305招商银行201501建设银行']
2 ['中信34K（淘宝V、i白、lu.xu.ry）']


## 实体识别
### 导入数据集
根据[PaddleHub文档的要求](https://paddlehub.readthedocs.io/zh_CN/release-v2.1/finetune/customized_dataset.html#id20),需要继承基类SeqLabelingDataset。按要求修改即可。

In [13]:
from paddlehub.datasets.base_nlp_dataset import SeqLabelingDataset


class MyDataset(SeqLabelingDataset):
    # 数据集存放目录
    base_path = 'data/data122751'
    # 数据集的标签列表
    label_list = ['B-BANK', 'I-BANK', 'B-PRODUCT', 'I-PRODUCT',
                  'B-COMMENTS_N', 'I-COMMENTS_N', 'B-COMMENTS_ADJ', 'I-COMMENTS_ADJ', 'O']
    label_map = {idx: label for idx, label in enumerate(label_list)}
    # 数据文件使用的分隔符
    split_char = '\002'

    def __init__(self, tokenizer, max_seq_len: int = 128, mode: str = 'train'):
        if mode == 'train':
            data_file = 'ner_train_data.tsv'
        elif mode == 'test':
            data_file = 'ner_test_data.tsv'
        else:
            data_file = 'ner_dev_data.tsv'
        super().__init__(
            base_path=self.base_path,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            data_file=data_file,
            label_file=None,
            label_list=self.label_list,
            split_char=self.split_char,
            is_file_with_header=False)

### 设置参数并训练
使用Adam优化器，相关默认设置见[文档说明](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/optimizer/Adam_cn.html)。

如要重新训练，请务必**指定一空目录保存参数**，或**删除默认的`ernie_checkpoint_ner`目录**，否则程序会用上一个epoch的参数继续训练！！！

In [14]:
num_epoch = 3  # Number of epoches for fine-tuning.
use_gpu = True  # help="Whether use GPU for fine-tuning, input should be True or False
learning_rate = 5e-5  # Learning rate used to train with warmup.
max_seq_len = 128  # Number of words of the longest seqence.
batch_size = 32  # Total examples' number in batch for training.
checkpoint_dir = './ernie_checkpoint_ner'  # help="Directory to model checkpoint
save_interval = 1  # Save checkpoint every n epoch.


# 选择模型、任务和类别数
model = hub.Module(name='ernie', task='token-cls',
                   label_map=MyDataset.label_map)

train_dataset = MyDataset(tokenizer=model.get_tokenizer(),
                          max_seq_len=max_seq_len, mode='train')
dev_dataset = MyDataset(tokenizer=model.get_tokenizer(),
                        max_seq_len=max_seq_len, mode='dev')
test_dataset = MyDataset(tokenizer=model.get_tokenizer(),
                         max_seq_len=max_seq_len, mode='test')

optimizer = paddle.optimizer.Adam(
    learning_rate=learning_rate, parameters=model.parameters())
trainer = hub.Trainer(
    model, optimizer, checkpoint_dir=checkpoint_dir, use_gpu=use_gpu)
trainer.train(train_dataset, epochs=num_epoch, batch_size=batch_size, eval_dataset=dev_dataset,
              save_interval=save_interval)
# 在测试集上评估当前训练模型
trainer.evaluate(test_dataset, batch_size=batch_size)

[2022-01-15 16:55:48,912] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-01-15 16:55:50,704] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-01-15 16:55:53,144] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-01-15 16:55:53,291] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2022-01-15 16:55:53,432] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...
[2022-01-15 16:55:53,621] [ WARNING] - Compatibility Warning: The params of ChunkEvaluator.compute has been modified. The old version is `inputs`, `lengths`, `predictions`, `labels` while the current version is `lengths`, `predictions`, `labels`.  Please update the usage.
[2022-01-15 16:55:55,848] [   TRAIN] - Epoch=1/3, Step=10/179 loss=0.2755 f1_score=0.0050 lr=0.000050 step/sec=4.14 | ETA 00:02:09
[2022-01-15 16:55:58,074] [   TRAIN] - Epoch=1/3, Step=20/179 loss=0.1296 f1_sc

{'metrics': defaultdict(int, {'f1_score': 0.8038293033840759})}

以上代码块如有运行问题，可直接运行funetune_ernie-ner.py源文件

### 结果预测
如要对验证集的进行预测并对其指标进行计算，可对file_path进行修改，并自行增加指标计算。由于训练过程中的指标已是F1，与评测指标相同，实际无需再自行计算比较。

我在本任务的训练中得到的最好的模型参数保存在`/ernie_checkpoint_ner_9505_2_f177.19/best_model/model.pdparams`以及`/ernie_checkpoint_ner_9505_3_f180.38/best_model/model.pdparams`（由于AI Studio平台限制，公开文件大小不超过1G，参数文件可能会在其他平台公开）

In [15]:
file_path = "data/data122751/ner_finaltest_data.tsv"
text = pd.read_csv(file_path, sep="\t", header=None)
label_list = ['B-BANK', 'I-BANK', 'B-PRODUCT', 'I-PRODUCT',
              'B-COMMENTS_N', 'I-COMMENTS_N', 'B-COMMENTS_ADJ', 'I-COMMENTS_ADJ', 'O']
label_map = {idx: label for idx, label in enumerate(label_list)}
data = [[i] for i in text[1]]

model = hub.Module(name='ernie', task='token-cls',
                   load_checkpoint='./ernie_checkpoint_ner/best_model/model.pdparams', label_map=label_map)
results = model.predict(data, max_seq_len=128, batch_size=1, use_gpu=True)

[2022-01-15 16:59:56,887] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-01-15 17:00:07,114] [    INFO] - Loaded parameters from /home/aistudio/ernie_checkpoint_ner/best_model/model.pdparams
[2022-01-15 17:00:07,236] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


部分结果展示，并保存到文件。

In [16]:
# print(data[0])
# print(results[0])

resultlist = []
for idx, text in enumerate(data):
    resultlist.append(" ".join(results[idx][1:len(text[0])+1]))
    if idx < 10:
        labels = results[idx][1:len(text[0])+1]
        print(
            f'Data: {text} \n Label: {", ".join(results[idx][1:len(text[0])+1])}\n')

resultsDF = pd.DataFrame(data=resultlist)
resultsDF.to_csv('ner_result.csv', sep='\t', header=None,
                 index=False, columns=None, mode="w")

Data: [['不', '能', '?', '?', '好', '像', '房', '贷', '跟', '信', '用', '卡', '是', '分', '开', '审', '核', '的', '反', '正', '我', '的', '不', '得']] 
 Label: O, O, O, O, O, O, B-PRODUCT, I-PRODUCT, O, B-PRODUCT, I-PRODUCT, I-PRODUCT, O, O, O, O, O, O, O, O, O, O, O, O

Data: [['我', '感', '觉', '这', '样', '才', '是', '合', '理', '的', '，', '花', '呗', '白', '条', '没', '要', '那', '么', '多', '信', '息', '，', '照', '样', '可', '以', '给', '额', '度', '。', '有', '征', '信', '威', '慑', '，', '没', '那', '么', '多', '人', '敢', '借', '了', '不', '还', '。', '与', '其', '眉', '毛', '胡', '子', '一', '把', '抓', '，', '还', '不', '如', '按', '额', '度', '区', '间', '对', '客', '户', '进', '行', '不', '同', '程', '度', '的', '调', '查', '，', '免', '.', '.', '.']] 
 Label: O, O, O, O, O, O, O, B-COMMENTS_ADJ, I-COMMENTS_ADJ, O, O, B-PRODUCT, I-PRODUCT, B-PRODUCT, I-PRODUCT, O, O, O, O, O, O, O, O, O, O, O, O, O, B-COMMENTS_N, I-COMMENTS_N, O, O, B-PRODUCT, I-PRODUCT, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-COMMENTS_N, I-COMMENTS_N, O, O, O

## 可能的提升改进方法

因为时间仓促，参考的资料较少，也没有很多时间进行调参和优化，这一模型的很多参数都使用了默认值。如果需要进一步优化，一方面可以通过调整学习率等参数，另一方面可以[相关文档和教程](https://aistudio.baidu.com/aistudio/projectdetail/2463239)进一步选用ERNIE-Gram，或加入条件随机场（CRF）等方式来提升序列标注的效果。

另外，从多次实验中可以发现，由于数据集分割是随机打乱后进行的，训练效果也受到数据集的制约。如果能寻找到一个好的数据集划分，也可以训练得到一个好的模型，但这是可遇而不可求的。